# Tavily 搜索

[Tavily 的 Search API](https://tavily.com) 是一个专为 AI 代理 (LLMs) 构建的搜索引擎，能够快速提供实时、准确、事实性的结果。

## 概览

### 集成详情
| 类 | 包 | 可序列化 | [JS 支持](https://js.langchain.com/docs/integrations/tools/tavily_search) |  最新包 |
|:--------------------------------------------------------------|:---------------------------------------------------------------| :---: | :---: | :---: |
| [TavilySearch](https://github.com/tavily-ai/langchain-tavily) | [langchain-tavily](https://pypi.org/project/langchain-tavily/) | ✅ | ✅  |  ![PyPI - Version](https://img.shields.io/pypi/v/langchain-tavily?style=flat-square&label=%20) |

### 工具特色
| [返回工件](/docs/how_to/tool_artifacts/) | 原生异步 |                       返回数据                        | 定价 |
| :---: | :---: |:--------------------------------------------------------:| :---: |
| ❌ | ✅ | title, URL, content snippet, raw_content, answer, images | 每月 1,000 次免费搜索 |


## 设置

该集成位于 `langchain-tavily` 包中。

In [ ]:
%pip install -qU langchain-tavily

### 凭证

我们还需要设置 Tavily API 密钥。你可以访问 [此网站](https://app.tavily.com/sign-in) 并创建一个账户来获取 API 密钥。

In [2]:
import getpass
import os

if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Tavily API key:\n")

## 实例化

在此，我们将展示如何实例化 Tavily 搜索工具。该工具接受各种参数来自定义搜索。实例化后，我们将使用一个简单的查询来调用该工具。此工具允许您通过 Tavily 的 Search API 端点完成搜索查询。

实例化
该工具在实例化时接受多种参数：

- max_results (可选, int): 返回搜索结果的最大数量。默认为 5。
- topic (可选, str): 搜索的类别。可以是 "general"（通用）、"news"（新闻）或 "finance"（财经）。默认为 "general"。
- include_answer (可选, bool): 在结果中包含对原始查询的回答。默认为 False。
- include_raw_content (可选, bool): 包含每个搜索结果的已清理和解析的 HTML。默认为 False。
- include_images (可选, bool): 在响应中包含查询相关图片的列表。默认为 False。
- include_image_descriptions (可选, bool): 包含每张图片的描述性文字。默认为 False。
- search_depth (可选, str): 搜索深度，可以是 "basic"（基本）或 "advanced"（高级）。默认为 "basic"。
- time_range (可选, str): 从当前日期回溯以过滤结果的时间范围 - "day"（天）、"week"（周）、"month"（月）或 "year"（年）。默认为 None。
- include_domains (可选, List[str]): 要特别包含的域名列表。默认为 None。
- exclude_domains (可选, List[str]): 要特别排除的域名列表。默认为 None。

有关可用参数的全面概述，请参阅 [Tavily Search API 文档](https://docs.tavily.com/documentation/api-reference/endpoint/search)。

In [ ]:
from langchain_tavily import TavilySearch

tool = TavilySearch(
    max_results=5,
    topic="general",
    # include_answer=False,
    # include_raw_content=False,
    # include_images=False,
    # include_image_descriptions=False,
    # search_depth="basic",
    # time_range="day",
    # include_domains=None,
    # exclude_domains=None
)

## 调用

### [直接用参数调用](/docs/concepts/tools)

Tavily 搜索工具在调用时接受以下参数：
- `query` (必需)：自然语言搜索查询
- 以下参数也可以在调用时设置：`include_images`、`search_depth`、`time_range`、`include_domains`、`exclude_domains`、`include_images`
- 出于可靠性和性能原因，某些影响响应大小的参数在调用时无法修改：“include_answer”和“include_raw_content”。这些限制可以防止意外的上下文窗口问题并确保结果的一致性。

注意：可选参数供代理动态设置，如果您在实例化过程中设置了某个参数，然后在调用工具时使用不同的值，则该工具将使用您在调用过程中传递的值。

In [ ]:
tool.invoke({"query": "What happened at the last wimbledon"})

### [使用 ToolCall 调用](/docs/concepts/tools)

我们也可以使用模型生成的 ToolCall 来调用工具，在这种情况下会返回一个 ToolMessage：

In [5]:
# This is usually generated by a model, but we'll create a tool call directly for demo purposes.
model_generated_tool_call = {
    "args": {"query": "euro 2024 host nation"},
    "id": "1",
    "name": "tavily",
    "type": "tool_call",
}
tool_msg = tool.invoke(model_generated_tool_call)

# The content is a JSON string of results
print(tool_msg.content[:400])

{"query": "euro 2024 host nation", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "UEFA Euro 2024 - Wikipedia", "url": "https://en.wikipedia.org/wiki/UEFA_Euro_2024", "content": "Tournament details Host country Germany Dates 14 June – 14 July Teams 24 Venue(s) 10 (in 10 host cities) Final positions Champions Spain (4th title) Runners-up England Tournament statisti


## 在 Agent 中使用

我们可以通过将工具绑定到 Agent 来直接在 Agent Executor 中使用我们的工具。这使得 Agent 能够动态地为 Tavily 搜索工具设置可用参数。

在下面的示例中，当我们要求 Agent 查找“哪个国家主办了 2024 年欧洲杯？仅包括维基百科来源。”时，Agent 将动态设置参数并调用 Tavily 搜索工具：调用 `tavily_search` 并使用 `{'query': 'Euro 2024 host nation', 'include_domains': ['wikipedia.org']`

import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs customVarName="llm" />

In [10]:
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("OPENAI_API_KEY:\n")

In [1]:
# | output: false
# | echo: false

# !pip install -qU langchain langchain-openai
from langchain.chat_models import init_chat_model

llm = init_chat_model(model="gpt-4o", model_provider="openai", temperature=0)

我们需要安装 langgraph：

In [ ]:
%pip install -qU langgraph

In [2]:
from langchain_tavily import TavilySearch
from langgraph.prebuilt import create_react_agent

# Initialize Tavily Search Tool
tavily_search_tool = TavilySearch(
    max_results=5,
    topic="general",
)

agent = create_react_agent(llm, [tavily_search_tool])

user_input = "What nation hosted the Euro 2024? Include only wikipedia sources."

for step in agent.stream(
    {"messages": user_input},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What nation hosted the Euro 2024? Include only wikipedia sources.
================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_yxmR4K2uadsQ8LKoyi8JyoLD)
 Call ID: call_yxmR4K2uadsQ8LKoyi8JyoLD
  Args:
    query: Euro 2024 host nation
    include_domains: ['wikipedia.org']
================================= Tool Message =================================
Name: tavily_search

{"query": "Euro 2024 host nation", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "UEFA Euro 2024 - Wikipedia", "url": "https://en.wikipedia.org/wiki/UEFA_Euro_2024", "content": "Tournament details Host country Germany Dates 14 June – 14 July Teams 24 Venue(s) 10 (in 10 host cities) Final positions Champions Spain (4th title) Runners-up England Tournament statistics Matches played 51 Goals scored 117 (2.29 per match) Attendance 2,681,288 (52,574 per m

## API 参考

有关 Tavily Search API 所有功能和配置的详细文档，请访问 API 参考：https://docs.tavily.com/documentation/api-reference/endpoint/search